# Testing

We now have a fully automated script! 🎉👏🏻🦄

The next step is to include **tests**... in fact testing should be a core part of our development process. In fact all of our **reproducible workflows** are analogous to experimental design in the scientific world

![science](./assets/the_difference.png)

<small> https://xkcd.com/242/ </small>

There are various approaches to tests software:
- Assertions
- Exceptions: within the code serve as ⚠️
- Unit tests: investigate the behaviour of units of code (e.g functions)
- Regression tests: defends against 🐛
- Integration tests: ⚙️ checks that the pieces work together as expected

We will start by testing some of our functions:
Open `03_country-subset.py` and add the following function:
    
```python 
def get_mean_price(filename):
    """ function to get the mean price of the wines
    rounded to 4 decimals"""
    wine = pd.read_csv(filename)
    mean_price = wine['price'].mean()
    return round(mean_price, 4)
```

And we will modify this function too:
```python
def get_country(filename, country):
   

    # Load table
    wine = pd.read_csv(filename)

    # Use the country name to subset data
    subset_country = wine[wine['country'] == country ].copy()

    # Subset the

    # Constructing the fname
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    fname = f'data/processed/{today}-winemag_{country}.csv'

    # Saving the csv
    subset_country.to_csv(fname)
    print(fname)  # print the fname from here

    return(subset_country)  #returns the data frame
```

Now we need to create out testing scripts. 
Some resources:
- Pytest usage examples can be found [here](http://doc.pytest.org/en/latest/usage.html)
- Rules for [test discovery](http://doc.pytest.org/en/latest/goodpractices.html)

Now we can create our tests:
```
$ touch tests/__init__.py
$ touch test_03_country_subset.py
```
Your test scrips should start with `test`

Your test script should look like this:
``` python
import importlib

country = importlib.import_module('.data.03_country-subset', 'src')

interim_data = "data/interim/2018-04-30-winemag_priceGBP.csv"
processed_data = "data/processed/2018-04-30-winemag_Chile.csv"

def test_get_mean_price():
    mean_price = country.get_mean_price(processed_data)
    assert mean_price == 20.7865
```

And you can run it from the shell using:
```
$ python -m pytest tests/test_03_country-subset.py
```

## What if you want all the decimal numbers?

``` python
import importlib
import numpy.testing as npt

country = importlib.import_module('.data.03_country-subset', 'src')

interim_data = "data/interim/2018-04-30-winemag_priceGBP.csv"
processed_data = "data/processed/2018-04-30-winemag_Chile.csv"

def test_get_mean_price():
    mean_price = country.get_mean_price(processed_data)
    assert mean_price == 20.7865
    npt.assert_allclose(country.get_mean_price(processed_data) , 20.787, rtol = 0.01)
```

The `numpy.testing.assert_allclose` allows you to set a tolerance 

### What else could go wrong?

What if we created a data set and we want to make sure that my interim or raw data has not changed? -> Thus my dataframes have not changes either?

```python 
import pandas.testing as pdt
import pandas as pd


interim_data = "data/interim/2018-05-09-winemag_priceGBP.csv"
processed_data = "data/processed/2018-05-09-winemag_Chile.csv"

def test_get_country():
    # call the function
    df = country.get_country(interim_data, 'Chile')
    
    # load my previous dataset
    base = pd.read_csv(processed_data)
    
    # check if I am getting a dataframe
    assert isinstance(df, pd.DataFrame)
    assert isinstance(base, pd.DataFrame)
    
    # check that they are the same dataframes
    pdt.assert_frame_equal(df, base)
```    

### See what we did in the previous steps?

We tested each of the functions in our module...
we did *unit testing*!
Notice something in the functions we just wrote? 
- Set-up: `mean = country.get_mean(interim_data)`
- Assertions: `assert mean_price == 20.786`

In [1]:
from IPython.core.display import HTML


def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()